# jupyterlabをsingularityコンテナで起動する
## 1.目的
- Jupyterlabコンテナをサーバで起動し、リモートのMBPでjupyterlabを使う
- コンテナは、dockerも作成するが、日常の仕様はsingularityをする。
- 今回は、家庭内のLAN環境内での使用とする。

## 2. 使用環境
### (1)サーバ（HP-Z820）
- GPU0: Geforce GTX 1080
- GPU1: Quadro K2000
- driver ver.: 440.95.01
- Ubuntu 18.04.4 LTS

### (2)クライアント（Mac Book Pro）
- macOS Catalina 10.15.5

## 3. 参考ページ
[この記事](https://qiita.com/radiol/items/48909d69ba8114edcbf2)を参考にした。 大変参考になった、感謝。

## 4. 手順
### (1) 作業用ディレクトリを作成、必要ファイルの準備
```
$ mkdir pytorch-lab
$ cd pytorch-lab
```
上記の参考記事を参考に、Dockerfileをこのディレクトリに作成。ベースとなるイメージは、ドライバーバージョンに合う比較的に新しいCUDAバージョンアップのpytorchを選択した。以下がDockerfileの内容。


```$ cat Dockerfile
FROM pytorch/pytorch:1.5.1-cuda10.1-cudnn7-runtime

# Install required libraries
RUN conda config --add channels pytorch \
 && conda config --append channels conda-forge \
 && conda update --all --yes --quiet \
 && conda install --yes --quiet \
    ipywidgets \
    jupyterlab \
    matplotlib \
    nodejs \
    opencv \
    pandas \
    scikit-learn \
    seaborn \
    sympy \
 && conda clean --all -f -y

# Install jupyter extensions
RUN jupyter nbextension enable --py --sys-prefix widgetsnbextension \
 && jupyter labextension install @jupyter-widgets/jupyterlab-manager

COPY jupyter_notebook_config.py /root/.jupyter/

```

同じく、上記の記事を参考に、jupyter_notebook_config.pyを作業用ディレクトリに配置。  

記事のとおり、以下の2行を追加した。
c.NotebookApp.allow_root = True
c.NotebookApp.password = 'sha1:・・・'


「sh1:」以下のハッシュ値は、次のようにして求めた。　自分の環境には、ipythonが入っていなかったので、先ずはipythonをインストール。
```
$ sudo apt install ipython
```

次に、ハッシュ値を得て、jupyter_notebook_config.pyに挿入した。
```
$ ipython

In [1]: from IPython.lib import passwd

In [2]: passwd()
Enter password:
Verify password:
Out[2]: 'sha1:ここにハッシュ値が表示されている'
```

```
# Copyright (c) Jupyter Development Team.
# Distributed under the terms of the Modified BSD License.

from jupyter_core.paths import jupyter_data_dir
import subprocess
import os
import errno
import stat

c = get_config()
c.NotebookApp.ip = '0.0.0.0'
c.NotebookApp.port = 8888
c.NotebookApp.open_browser = False
c.NotebookApp.allow_root = True
c.NotebookApp.password = 'sha1:ce60f68cef4d:5447853c136a04ffaa1c0a4d1fc616564d9941bc'

# https://github.com/jupyter/notebook/issues/3130
c.FileContentsManager.delete_to_trash = False

# Generate a self-signed certificate
if 'GEN_CERT' in os.environ:
    dir_name = jupyter_data_dir()
    pem_file = os.path.join(dir_name, 'notebook.pem')
    try:
        os.makedirs(dir_name)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(dir_name):
            pass
        else:
            raise

    # Generate an openssl.cnf file to set the distinguished name
    cnf_file = os.path.join(os.getenv('CONDA_DIR', '/usr/lib'), 'ssl', 'openssl.cnf')
    if not os.path.isfile(cnf_file):
        with open(cnf_file, 'w') as fh:
            fh.write('''\
[req]
distinguished_name = req_distinguished_name
[req_distinguished_name]
''')

    # Generate a certificate if one doesn't exist on disk
    subprocess.check_call(['openssl', 'req', '-new',
                           '-newkey', 'rsa:2048',
                           '-days', '365',
                           '-nodes', '-x509',
                           '-subj', '/C=XX/ST=XX/L=XX/O=generated/CN=generated',
                           '-keyout', pem_file,
                           '-out', pem_file])
    # Restrict access to the file
    os.chmod(pem_file, stat.S_IRUSR | stat.S_IWUSR)
    c.NotebookApp.certfile = pem_file

# Change default umask for all subprocesses of the notebook server if set in
# the environment
if 'NB_UMASK' in os.environ:
    os.umask(int(os.environ['NB_UMASK'], 8))
```

### (2) Dockerイメージの作成と確認
- Dockerイメージをビルドする
```
$ sudo docker build -t pytorch-lab ./
```


- Dockerイメージを起動
```
$ sudo docker run -d --rm --gpus all -v ~/ml:/workspace -p 8888:8888 --name pytorch pytorch-lab jupyter lab
```


- サーバ上の（ローカルの）ブラウザで、```http://localhost:8888```と入力し、接続する。
- パスワード入力「log in」を押下すると、jupyterが起動される。

### (3) singularityイメージの作成
- dockerイメージからsingularityイメージを作成

ローカルdockerファイルを指定する場合には、```docker-daemon:```とする。
```
$ sudo singularity build pytorch-lab.sif docker-daemon://pytorch-lab:latest
```

### (4) singularityコンテナの起動とローカルから接続確認
- 作成したsingularityイメージを起動する。
```
$ singularity exec --nv sifs/pytorch-lab.sif jupyter lab
```

- ローカルのブラウザから接続確認
ローカルサーバ上のブラウザで、```http://localhost:8888```と入力するご、パスワード入力画面が開いた。

パスワードを入力してもうまく行かず、TOKENを入力したらうまく行った。

### (5) リモートからの接続
- リモート（Mac Book Pro）で、jupyterlabを使う

以下は、MBP側での操作。

```
$ ssh -N -L 8888:localhost:8888 user名@サーバのIPアドレス
```

サーバのuser名に対するパスワードを入力する。

次に、MBPのブラウザで、```http://localhost:8888```と入力すると、接続できた。

ここでも、パスワードではなく、TOKENしか受け付けないようだ。

### (6) サーバでのsinguralityコンテナ起動コマンド
```$HOME/bin```には、PATHが通っており、ファイルには実行権が付いて前提で、以下のとおり。
```
$cat $HOME/bin/jupyterlab
cd $HOME/jupyterlab
export SINGULARITYENV_CUDA_VISIBLE_DEVICES=0
singularity exec --nv $HOME/sifs/pytorch-lab.sif jupyter lab
```

```$HOME/jupyterlab```に作成したノートが保存される。